# In this notebook, we'll take some first steps with MLlib, Spark's machine learning library


http://spark.apache.org/docs/latest/mllib-guide.html

## Spark has two ML libraries to enable machine learning at scale:
### It divides into two packages:

* spark.mllib contains the original API built on top of RDDs.
* spark.ml provides higher-level API built on top of DataFrames for constructing ML pipelines.

### We will try to predict user ratings based on other user ratings (collaborative filtering!). More specifically, we will:
* Load user ratings data,
* Separate it into training and test sets,
* Train an Alternating Lease Squares model,
* Make predictions on the test set
* Compare the prediction to truth,
* Produce a quantitative goodness-of-fit 

In [ ]:
from pyspark.mllib.recommendation import ALS
from pyspark.mllib.recommendation import Rating

#### Create a DataFrame from the csv while repartitioning and persisting

In [ ]:
# Read in the ratings file (fromUserId, toUserId, rating).  These ratings are 0-9.
ratings_raw_DF = sqlContext.read.format("com.databricks.spark.csv") \
                           .options(header="false") \
                           .load("s3a://insight-spark-after-dark/ratings.csv.gz") \
                           .repartition(1000)

To write SparkSQL, we need to create a table object from our dataframe which we can use to run SparkSQL commands. The transformation registerTempTable() does this and we call our table 'ratings_raw_tbl'.

In [ ]:
# Register the ratings_raw DataFrame as a temp table - this allows us to run the SparkSQL queries
ratings_raw_DF.registerTempTable("ratings_raw_tbl")

In [ ]:
#let's trigger the job and load the data
#just take a sample for now to save time
#ratings_raw_DF = ratings_raw_DF.sample(False, .05, 20)
ratings_raw_DF.persist(StorageLevel.MEMORY_AND_DISK_SER).count()

In [ ]:
#let's look at it! (fromUserId, toUserId, rating)
ratings_raw_DF.take(5)

In [ ]:
# Cast the DataFrame to enforce a schema with (from_user_id, to_user_id, rating)
ratings_DF = sqlContext.sql("""
SELECT
    CAST(C0 as int) AS from_user_id,
    CAST(C1 as int) AS to_user_id,
    CAST(C2 as int) AS rating
FROM 
    ratings_raw_tbl
""")

In [ ]:
# Create mllib.recommendation.Rating RDD from ratings DataFrame
ratings_RDD = ratings_DF.rdd.map(lambda r: Rating(r.from_user_id, r.to_user_id, r.rating))

ratings_RDD.take(5)

### Task 1: Separate ratings data into training data (80%) and test data (20%)


In [ ]:
# Task 1: Separate ratings data into training data (80%) and test data (20%)
split_ratings_RDD = ratings_RDD.randomSplit([0.8, 0.2])
train_ratings_RDD = split_ratings_RDD[0]
test_ratings_RDD = split_ratings_RDD[1]

## One algorithm to learn the features in a collaborative filtering model is Alternating Least Squares. We learn factor vectors for each from_user and each to_user. Reach more here:
#### http://bugra.github.io/work/notes/2014-04-19/alternating-least-squares-method-for-collaborative-filtering/

In [ ]:
# Train the ALS model using the training data and various model hyperparameters
# hyperparameters are: 
# rank       number of features to use
# iterations number of iterations of ALS (recommended: 10-20
# lambda_     regularization factor (recommended: 0.01)
# blocks     level of parallelism to split computation into
# seed       random seed

model = ALS.train(train_ratings_RDD, 1, iterations=5, lambda_=0.01, blocks=10)

In [ ]:
# Convert known test data to have only (from_user_id, to_user_id), 
# we can then feed this into our model to predict new ratings for 
# pairs of from_user_ids and to_user_ids
test_from_to_RDD = test_ratings_RDD.map(lambda r: (r[0], r[1]))

### Task 2: Test the model by predicting the ratings for the test dataset
#### Hint: the predictAll() method can be called with the RDD as input to return a new RDD


In [ ]:
# Task 2: Test the model by predicting the ratings for the test dataset
test_predictions_RDD = model.predictAll(test_from_to_RDD)

test_predictions_RDD.take(5)

### Task 3: Prepare the known test predictions and actual predictions for comparison keyed by (from, to)
#### Hint: We'd like to have an RDD with the form of ((from, to), rating), think map().


In [ ]:
# Task 3: Prepare the known test predictions and actual predictions for comparison keyed by (from, to)
test_predictions_RDD = test_predictions_RDD.map(lambda r: ((r[0], r[1]), r[2]))
test_actual_RDD = test_ratings_RDD.map(lambda r: ((r[0], r[1]), r[2]))

### Task 4: Join the known test predictions with the actual predictions


In [ ]:
# Task 4: Join the known test predictions with the actual predictions
test_to_actual_ratings_RDD = test_actual_RDD.join(test_predictions_RDD)
test_to_actual_ratings_RDD.take(10)

### Task 5: Evaluate the model using Mean Absolute Error (MAE) between the known test ratings and the actual predictions 


In [ ]:
# Task 5: Evaluate the model using Mean Absolute Error (MAE) between the known test ratings and the actual predictions 
mean_absolute_rating_error = test_to_actual_ratings_RDD.map(lambda r: abs(r[1][0]-r[1][1]))\
                                                       .mean()

print mean_absolute_rating_error

# Next Steps

### Task 6: Instead of a training and test set only, construct a cross validation set and use it to optimize over a vector of possible regularization parameters


### Task 7: If you're considering applying machine learning at scale for tomorrow's project, explore MLlib and consider your options!
